In [1]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [2]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [3]:
devops_df = pd.read_csv('/content/devops_df.csv')
infosec_df = pd.read_csv('/content/infosec_df.csv')
prog_df = pd.read_csv('/content/prog_df.csv')
popsci_df = pd.read_csv('/content/popsci_df.csv')
ai_df = pd.read_csv('/content/ai_df.csv')
pm_df = pd.read_csv('/content/pm_df.csv')

df = pd.concat([devops_df, infosec_df, prog_df, popsci_df, ai_df, pm_df],ignore_index=True )

In [4]:
df = df[['category', 'link', 'text', 'title']]

In [5]:
df = df[df['text'] != 'Без текста']
df

,category,link,text,title
0,devops,https://habr.com/ru/companies/nixys/articles/8...,В современном мире Kubernetes уже стал промышл...,Спускаясь с облаков в ад: развёртывание Kubern...
1,devops,https://habr.com/ru/companies/sberbank/article...,Всем привет! Если в компании растёт количество...,Оптимизируем системные ресурсы при развёртыван...
2,devops,https://habr.com/ru/companies/ru_mts/articles/...,"Привет, Хабр! Меня зовут Филипп Бочаров, я рук...","Одна платформа, чтобы мониторить всех: как мы ..."
3,devops,https://habr.com/ru/companies/kuper/articles/8...,Всем привет! Меня зовут Роза и я MLOps-инженер...,Готовим по рецепту: CI/CD в MLOps
4,devops,https://habr.com/ru/companies/flant/articles/8...,"Всем привет! На связи Дмитрий Силкин, DevOps-и...",Обзор Kwasm — оператора для запуска WebAssembl...
...,...,...,...,...
5635,pm,https://habr.com/ru/articles/755992/,Расскажите о себе и сути проекта?Когда я тольк...,Как не забросить свой пет-проект. Мой кейс зап...
5636,pm,https://habr.com/ru/articles/755982/,"Всем доброго дня ! Недавно я оценил, как же эт...","Организовываем свою активность, через простран..."
5637,pm,https://habr.com/ru/articles/755936/,Всякую сущность следует анализировать в катего...,Понятие «сущность»
5638,pm,https://habr.com/ru/articles/755906/,С 1 сентября 2022 г. вступили в силу изменени...,Независимая оценка ГИПов: катастрофа или новая...


In [6]:
df_new = df[~df.text.isna()]

In [7]:
X = df_new[['text']]
y = df_new['category']

X.shape

(5606, 1)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = CountVectorizer()
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42, solver='newton-cg')
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(4484, 369191)
                         precision    recall  f1-score   support

artificial_intelligence       0.69      0.66      0.68       185
                 devops       0.78      0.76      0.77       191
           infosecurity       0.75      0.69      0.72       178
                     pm       0.77      0.86      0.81       191
        popular_science       0.79      0.80      0.80       187
            programming       0.64      0.65      0.65       190

               accuracy                           0.74      1122
              macro avg       0.74      0.74      0.74      1122
           weighted avg       0.74      0.74      0.74      1122



In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier

vec = CountVectorizer() # подбор гиперпараметров очень помогает
vec.fit(X_train['text'])

bow = vec.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = vec.transform(X_test['text'])

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = CatBoostClassifier(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)

print(classification_report(y_test, pred))

(4484, 369191)
Learning rate set to 0.08542
0:	learn: 1.7192399	total: 7.93s	remaining: 2h 11m 58s
1:	learn: 1.6521124	total: 12.9s	remaining: 1h 47m 33s
2:	learn: 1.5901895	total: 17.7s	remaining: 1h 38m 10s
3:	learn: 1.5287323	total: 24.3s	remaining: 1h 40m 55s
4:	learn: 1.4868636	total: 28.9s	remaining: 1h 35m 59s
5:	learn: 1.4420942	total: 33.7s	remaining: 1h 33m 2s
6:	learn: 1.4030998	total: 40.2s	remaining: 1h 35m 5s
7:	learn: 1.3736250	total: 44.9s	remaining: 1h 32m 46s
8:	learn: 1.3372558	total: 50.8s	remaining: 1h 33m 13s
9:	learn: 1.3088299	total: 56.2s	remaining: 1h 32m 45s
10:	learn: 1.2814304	total: 1m	remaining: 1h 31m 10s
11:	learn: 1.2574888	total: 1m 7s	remaining: 1h 32m 26s
12:	learn: 1.2323350	total: 1m 11s	remaining: 1h 31m 6s
13:	learn: 1.2124226	total: 1m 16s	remaining: 1h 30m 3s
14:	learn: 1.1931257	total: 1m 23s	remaining: 1h 30m 51s
15:	learn: 1.1794640	total: 1m 27s	remaining: 1h 29m 46s
16:	learn: 1.1646693	total: 1m 33s	remaining: 1h 29m 53s
17:	learn: 1.146

In [10]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [14]:
vec = CountVectorizer() # подбор гиперпараметров очень помогает
vec.fit(X['text'])

bow = vec.transform(X['text'])  # bow — bag of words (мешок слов)

print(bow.shape)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)

clf = CatBoostClassifier(random_state=42)
clf.fit(bow, y)

(5606, 423716)
Learning rate set to 0.086398
0:	learn: 1.7049667	total: 10.4s	remaining: 2h 53m 35s
1:	learn: 1.6334647	total: 17.8s	remaining: 2h 27m 50s
2:	learn: 1.5741162	total: 23.3s	remaining: 2h 9m 7s
3:	learn: 1.5298586	total: 30.4s	remaining: 2h 6m 8s
4:	learn: 1.4760489	total: 35.9s	remaining: 1h 59m 13s
5:	learn: 1.4354023	total: 41.7s	remaining: 1h 55m 3s
6:	learn: 1.3912622	total: 48.6s	remaining: 1h 54m 55s
7:	learn: 1.3609102	total: 54s	remaining: 1h 51m 36s
8:	learn: 1.3253645	total: 1m 1s	remaining: 1h 52m 17s
9:	learn: 1.2968835	total: 1m 6s	remaining: 1h 50m 8s
10:	learn: 1.2701819	total: 1m 14s	remaining: 1h 50m 56s
11:	learn: 1.2450959	total: 1m 19s	remaining: 1h 49m 14s
12:	learn: 1.2212519	total: 1m 25s	remaining: 1h 48m 42s
13:	learn: 1.2030098	total: 1m 32s	remaining: 1h 48m 26s
14:	learn: 1.1823063	total: 1m 37s	remaining: 1h 47m
15:	learn: 1.1627133	total: 1m 44s	remaining: 1h 47m 33s
16:	learn: 1.1414656	total: 1m 50s	remaining: 1h 46m 22s
17:	learn: 1.12944

In [15]:
import joblib
joblib.dump(clf, "trained_model.pkl")

['trained_model.pkl']